## ZAD1

In [401]:
import re
import random as rnd
from collections import defaultdict
import numpy as np
import glob
from pprint import pprint
from itertools import permutations
from functools import reduce
import pprint

In [359]:
def generate_data(data):
    RE = re.compile(r'[a-zA-ZęóąśłżźćńĘÓĄŚŁŻŹĆŃ\-]+')
    percent_train = 0.9
    normalized_data = [[word.lower() for word in re.findall(RE, line)] for line in data]
    normalized_data = [superbase_mapping(sentence) for sentence in normalized_data if sentence]
    randomized = normalized_data.copy()
    rng = rnd.Random()
    rng.shuffle(randomized)
    train_data = randomized[:int(percent_train * len(randomized))]
    validate_data = randomized[-int((1.0 - percent_train) * len(randomized)):]
    return train_data, validate_data, randomized, normalized_data

def calculate_chance_of_unknown(train, validate):
    dict_words = {}
    for line in train:
        for word in line:
            if word not in dict_words:
                dict_words[word] = 0
            dict_words[word] += 1
    not_found = 0
    count = 0
    for line in validate:
        for word in line:
            count += 1
            if word not in dict_words:
                not_found += 1
    return(not_found / count)

def calculate_corpus_sentence(file):
    with open (file) as corpus:
        data = corpus.readlines()
    train_data, validate_data, random, normalized_data = generate_data(data)
    bgram = digram(normalized_data)
    tgram = trigram(normalized_data)
    chance = calculate_chance_of_unknown(train_data, validate_data)
    dict_word = {}
    dict_character = {}
    flat = [item for sublist in normalized_data for item in sublist]
    for w in flat:
        if w not in dict_word:
            dict_word[w] = 0
        dict_word[w] += 1
        for c in w:
            if c not in dict_character:
                dict_character[c] = 0
            dict_character[c] += 1
    for k in dict_word:
        dict_word[k] = np.log(dict_word[k] / len(flat))
    char_num = sum([dict_character[k] for k in dict_character])
    for k in dict_character:
        dict_character[k] = np.log(dict_character[k] / char_num)
    return dict_character, dict_word, chance, bgram, tgram
    
    

In [110]:
super_dict = {}
def superbase_mapping(word_list):
    if len(super_dict) == 0:
        with open('../train data/superbazy.txt') as bazy:
            for line in bazy:
                w1, w2 = (line.split(' ')[0], line.split(' ')[1])
                super_dict[w1] = w2.strip()
    return [super_dict[w] if w in super_dict else w for w in word_list]

In [340]:
def digram(text):
    grams = defaultdict(float)
    total = 0
    for line in text:
        if not line:
            continue
        start = line[0]
        for w1, w2 in zip(line, line[1:]):
            grams[(w1, w2)] += 1
            total += 1
            start = word
    
    for gram in grams:
        grams[gram] = np.log(grams[gram] / total)
    return grams

In [341]:
def trigram(text):
    grams = defaultdict(float)
    total = 0
    for line in text:
        if not line:
            continue
        for w1, w2, w3 in zip(line, line[1:], line[2:]):
            grams[(w1, w2, w3)] += 1
            total += 1
            start = word
    
    for gram in grams:
        grams[gram] = np.log(grams[gram] / total)
    return grams

In [377]:
ochar_dict, oword_dict, ochance_of_unknown, odig, otri = calculate_corpus_sentence('../train data/dane_pozytywistyczne/korpus_orzeszkowej.txt')


In [378]:
pchar_dict, pword_dict, pchance_of_unknown, pdig, ptri = calculate_corpus_sentence('../train data/dane_pozytywistyczne/korpus_prusa.txt')

In [379]:
schar_dict, sword_dict, schance_of_unknown, sdig, stri = calculate_corpus_sentence('../train data/dane_pozytywistyczne/korpus_sienkiewicza.txt')

In [524]:
def naive_bayes_test(test_data, A=0.2, B=0.3, C=0.4):
        Names = {
            'Orzeszkowa': (ochar_dict, oword_dict, odig, otri),
            'Prus' : (pchar_dict, pword_dict, pdig, ptri),
            'Sienkiewicz': (schar_dict, sword_dict, sdig, stri)
        }
        D = 1 - A - B - C
        char_prop = defaultdict(float)
        word_prop = defaultdict(float)
        dig_prop = defaultdict(float)
        tri_prop = defaultdict(float)
        for name, data in Names.items():
            char_dict, word_dict, dig, tri = data
            for sentence in test_data:
                for w1, w2, w3 in zip(sentence, sentence[1:], sentence[2:]):
                    c_prop = 0
                    for c in w1:
                        c_prop += D * char_dict[c] if c in char_dict else np.log(1/1000000)
                    char_prop[name] += c_prop / len(w1)
                    word_prop[name] += word_dict[w1] if w1 in word_dict else np.log(1/110000)
                    dig_prop[name] += dig[(w1, w2)] if (w1, w2) in dig else np.log(1/100000)
                    tri_prop[name] += tri[(w1, w2, w3)] if (w1, w2, w3) in tri else np.log(1/50000)
        for name in Names:
            char_prop[name] = -1 / char_prop[name]
            word_prop[name] = -1 / word_prop[name]                  
            dig_prop[name] = -1 / dig_prop[name]
            tri_prop[name] = -1 / tri_prop[name]

        s_char = sum(char_prop.values())
        s_word = sum(word_prop.values())
        s_dig = sum(dig_prop.values())
        s_tri = sum(tri_prop.values())
        
        s_sum = s_char + s_word + s_dig + s_tri
            
        for name in Names:
            char_prop[name] *= 1 / s_char
            word_prop[name] *= 1 / s_word
            dig_prop[name] *= 1 / s_dig
            tri_prop[name] *= 1 / s_tri
        
        results = {}
        for author in Names:
            results[author] = (s_char / s_sum) * char_prop[author] + (s_word / s_sum) * word_prop[author] + (s_dig / s_sum) * dig_prop[author] + (s_tri / s_sum) * dig_prop[author]
        return results

In [525]:
RE = re.compile(r'[a-zA-ZęóąśłżźćńĘÓĄŚŁŻŹĆŃ\-]+')
res = []
for path in glob.glob('../train data/dane_pozytywistyczne/testy1/*'):
    filename = path.split('/')[-1]
    with open(path, 'rt') as test:
        normalized_data = [[word.lower() for word in re.findall(RE, line)] for line in test]
        normalized_data = [superbase_mapping(sentence) for sentence in normalized_data if sentence]
    output = naive_bayes_test(normalized_data, A=0.25, B=0.3, C=0.35)
    res.append((filename, max(output.items(), key=lambda t: t[1])[0]))
res.sort(key=lambda t: t[0])
pprint(res)

[('test_orzeszkowej.txt', 'Orzeszkowa'),
 ('test_orzeszkowej1.txt', 'Orzeszkowa'),
 ('test_orzeszkowej11.txt', 'Sienkiewicz'),
 ('test_orzeszkowej13.txt', 'Orzeszkowa'),
 ('test_orzeszkowej15.txt', 'Orzeszkowa'),
 ('test_orzeszkowej17.txt', 'Orzeszkowa'),
 ('test_orzeszkowej19.txt', 'Orzeszkowa'),
 ('test_orzeszkowej21.txt', 'Orzeszkowa'),
 ('test_orzeszkowej3.txt', 'Orzeszkowa'),
 ('test_orzeszkowej5.txt', 'Orzeszkowa'),
 ('test_orzeszkowej7.txt', 'Orzeszkowa'),
 ('test_orzeszkowej9.txt', 'Orzeszkowa'),
 ('test_prusa0.txt', 'Sienkiewicz'),
 ('test_prusa10.txt', 'Prus'),
 ('test_prusa12.txt', 'Prus'),
 ('test_prusa14.txt', 'Prus'),
 ('test_prusa16.txt', 'Prus'),
 ('test_prusa18.txt', 'Prus'),
 ('test_prusa2.txt', 'Prus'),
 ('test_prusa20.txt', 'Prus'),
 ('test_prusa22.txt', 'Prus'),
 ('test_prusa24.txt', 'Prus'),
 ('test_prusa26.txt', 'Prus'),
 ('test_prusa28.txt', 'Prus'),
 ('test_prusa30.txt', 'Prus'),
 ('test_prusa32.txt', 'Prus'),
 ('test_prusa34.txt', 'Prus'),
 ('test_prusa36.txt'

In [526]:
(sum([1 for x in res if x[0].split('_')[1][0] == x[1].lower()[0]]), len(res))

(47, 60)

## zad2

In [446]:
zad2_bigram_dict = {}
tag_dict = defaultdict(dict)
tag_mapper= {}

In [457]:
def generate_bigram_of(list_of_lists):
    words = set(reduce(list.__add__, list_of_lists))
    dict = {}
    with open('../train data/2grams', 'r', encoding='utf8') as bigram:
        for line in bigram:
            spl = [l for l in line.strip().lower().split(' ') if l]
            occ, w1, w2 = int(spl[0]), spl[1], spl[2]
            if w1 not in words:
                continue
            if w1 in dict:
                _, l, _ = dict[w1][-1]
                dict[w1].append((l+1, occ + l, w2))
            else:
                dict[w1] = [(0, occ, w2)]
    return dict

def zad2(a=0.1):
    def perm_heu(permutation_list, dict):
        res = []
        mapped = permutation_list#superbase_mapping(permutation_list)
        for w1, w2 in zip(mapped, mapped[1:]):
            if not w1 in dict:
                continue
            else:
                asc = dict[w1]
                score = -0.1
                _, last, _ = asc[-1]
                for lr, hr, w in asc:
                    if w == w2:
                        score = (hr - lr + 1) / last
                res.append(score)
        return (' '.join(permutation_list), sum(res))
    
    def tag_heuristic(perm, dict):
        global tag_dict
        global tag_mapper
        #perm = superbase_mapping(perm)
        if not tag_dict:
            with open('../train data/supertags.txt', 'r', encoding='utf8') as supertags:
                for line in supertags:
                    line = line.strip()
                    tag_mapper[line.split(' ')[0].lower()] = line.split(' ')[1].strip()
            for k in dict:
                if k not in tag_mapper:
                    continue
                for lr, hr, w in dict[k]:
                    if w in tag_mapper:
                        ktag = tag_mapper[k]
                        if tag_mapper[w] not in tag_dict[ktag]:
                            tag_dict[ktag][tag_mapper[w]] = 0
                        tag_dict[ktag][tag_mapper[w]] += (hr - lr) + 1
        res = [-0.1]
        for w1, w2 in zip(perm, perm[1:]):
            if w1 not in tag_mapper or w2 not in tag_mapper:
                continue
            tag1 = tag_mapper[w1]
            tag2 = tag_mapper[w2]
            if tag2 not in tag_dict[tag1]:
                continue
            total = sum(tag_dict[tag1].values())
            res.append(tag_dict[tag1][tag2] / total)
        return sum(res)

    def heu_interp(h1, h2):
        return h1[0], a * h2 + (1 - a) * h1[1]
    
    test_data = [
        'Judyta dała wczoraj Stefanowi czekoladki',
        'Babuleńka miała dwa rogate koziołki',
        'Wczoraj wieczorem spotkałem pewną piękną kobietę',
        'Dlaczego zawsze mam największego pecha',
        'Po raz pierwszy zauważyłem ten szczegół',
        'Zjadłem pyszną kanapkę z szynką i serem'
    ]
    results = []
    global zad2_bigram_dict
    if len(zad2_bigram_dict) == 0:
        zad2_bigram_dict = generate_bigram_of([test.lower().split(' ') for test in test_data])
    for test in test_data:
        results.append((test, [heu_interp(perm_heu(perm, zad2_bigram_dict), tag_heuristic(perm, zad2_bigram_dict)) for perm in permutations(test.lower().split(' '))]))

    for test, perm_list in results:
        perm_list.sort(reverse=True, key=lambda x: x[1])
        print(u'test: {0}\n1* - {1}\n2* - {2}\n3* - {3}\n'.format(test, perm_list[0], perm_list[1], perm_list[2]))


In [454]:
zad2(a=0.5)

test: Judyta dała wczoraj Stefanowi czekoladki
1* - ('dała wczoraj stefanowi czekoladki judyta', -0.022933789763080453)
2* - ('wczoraj stefanowi dała czekoladki judyta', -0.022933789763080453)
3* - ('dała czekoladki wczoraj stefanowi judyta', -0.02633515030729814)

test: Babuleńka miała dwa rogate koziołki
1* - ('miała rogate koziołki dwa babuleńka', -0.09977864842729708)
2* - ('miała koziołki rogate dwa babuleńka', -0.09977864842729708)
3* - ('rogate miała koziołki dwa babuleńka', -0.09977864842729708)

test: Wczoraj wieczorem spotkałem pewną piękną kobietę
1* - ('wczoraj kobietę wieczorem pewną spotkałem piękną', 0.014881529456569658)
2* - ('wczoraj kobietę wieczorem pewną piękną spotkałem', 0.014881529456569658)
3* - ('wczoraj kobietę wieczorem piękną spotkałem pewną', 0.014881529456569658)

test: Dlaczego zawsze mam największego pecha
1* - ('mam pecha dlaczego zawsze największego', -0.03230853756627626)
2* - ('pecha dlaczego zawsze największego mam', -0.03230853756627626)
3* - ('ma

In [455]:
zad2(a=0.1)

test: Judyta dała wczoraj Stefanowi czekoladki
1* - ('dała wczoraj stefanowi czekoladki judyta', -0.004554568032084956)
2* - ('wczoraj stefanowi dała czekoladki judyta', -0.004554568032084956)
3* - ('dała czekoladki wczoraj stefanowi judyta', -0.005234840140928495)

test: Babuleńka miała dwa rogate koziołki
1* - ('miała rogate koziołki dwa babuleńka', -0.09995572968545943)
2* - ('miała koziołki rogate dwa babuleńka', -0.09995572968545943)
3* - ('rogate miała koziołki dwa babuleńka', -0.09995572968545943)

test: Wczoraj wieczorem spotkałem pewną piękną kobietę
1* - ('wczoraj kobietę wieczorem pewną spotkałem piękną', 0.003056780692641766)
2* - ('wczoraj kobietę wieczorem pewną piękną spotkałem', 0.003056780692641766)
3* - ('wczoraj kobietę wieczorem piękną spotkałem pewną', 0.003056780692641766)

test: Dlaczego zawsze mam największego pecha
1* - ('mam pecha dlaczego zawsze największego', -0.005450433490513594)
2* - ('pecha dlaczego zawsze największego mam', -0.005450433490513594)
3* - (

In [456]:
zad2(a=0.8)

test: Judyta dała wczoraj Stefanowi czekoladki
1* - ('judyta czekoladki stefanowi dała wczoraj', -0.03469387755102041)
2* - ('dała judyta czekoladki stefanowi wczoraj', -0.03469387755102041)
3* - ('dała czekoladki judyta stefanowi wczoraj', -0.03469387755102041)

test: Babuleńka miała dwa rogate koziołki
1* - ('miała rogate koziołki dwa babuleńka', -0.09964583748367534)
2* - ('miała koziołki rogate dwa babuleńka', -0.09964583748367534)
3* - ('rogate miała koziołki dwa babuleńka', -0.09964583748367534)

test: Wczoraj wieczorem spotkałem pewną piękną kobietę
1* - ('wczoraj kobietę wieczorem pewną spotkałem piękną', 0.02375009102951558)
2* - ('wczoraj kobietę wieczorem pewną piękną spotkałem', 0.02375009102951558)
3* - ('wczoraj kobietę wieczorem piękną spotkałem pewną', 0.02375009102951558)

test: Dlaczego zawsze mam największego pecha
1* - ('mam pecha dlaczego zawsze największego', -0.05245211562309827)
2* - ('pecha dlaczego zawsze największego mam', -0.05245211562309827)
3* - ('mam naj

In [458]:
zad2(a=0.5)

test: Judyta dała wczoraj Stefanowi czekoladki
1* - ('judyta dała wczoraj czekoladki stefanowi', -0.13278001633454298)
2* - ('stefanowi judyta dała wczoraj czekoladki', -0.13278001633454298)
3* - ('czekoladki judyta dała wczoraj stefanowi', -0.13392238135041507)

test: Babuleńka miała dwa rogate koziołki
1* - ('dwa rogate miała babuleńka koziołki', -0.0861065307630569)
2* - ('dwa rogate koziołki miała babuleńka', -0.0861065307630569)
3* - ('koziołki dwa rogate miała babuleńka', -0.0861065307630569)

test: Wczoraj wieczorem spotkałem pewną piękną kobietę
1* - ('wczoraj wieczorem spotkałem pewną piękną kobietę', 0.21896871047705163)
2* - ('pewną piękną kobietę wieczorem spotkałem wczoraj', 0.21861016962370375)
3* - ('spotkałem pewną piękną kobietę wieczorem wczoraj', 0.21615512132709372)

test: Dlaczego zawsze mam największego pecha
1* - ('dlaczego zawsze mam największego pecha', 0.05095256227126561)
2* - ('mam dlaczego zawsze największego pecha', 0.01400291690630695)
3* - ('największego

In [459]:
zad2(a=0.1)

test: Judyta dała wczoraj Stefanowi czekoladki
1* - ('stefanowi judyta dała wczoraj czekoladki', -0.1827959166646358)
2* - ('judyta dała wczoraj czekoladki stefanowi', -0.18279591666463582)
3* - ('czekoladki judyta dała wczoraj stefanowi', -0.1830243896678102)

test: Babuleńka miała dwa rogate koziołki
1* - ('babuleńka miała dwa rogate koziołki', -0.17620666599170376)
2* - ('koziołki babuleńka miała dwa rogate', -0.17620666599170376)
3* - ('babuleńka koziołki miała dwa rogate', -0.17688693810054731)

test: Wczoraj wieczorem spotkałem pewną piękną kobietę
1* - ('wczoraj wieczorem spotkałem pewną piękną kobietę', 0.06486619248120291)
2* - ('pewną piękną kobietę wieczorem spotkałem wczoraj', 0.052846353877496824)
3* - ('spotkałem pewną piękną kobietę wieczorem wczoraj', 0.05181390941594892)

test: Dlaczego zawsze mam największego pecha
1* - ('dlaczego zawsze mam największego pecha', 0.014037428121282267)
2* - ('największego pecha dlaczego mam zawsze', -0.07130263917704426)
3* - ('dlaczego

In [460]:
zad2(a=0.8)

test: Judyta dała wczoraj Stefanowi czekoladki
1* - ('judyta dała wczoraj czekoladki stefanowi', -0.09526809108697334)
2* - ('stefanowi judyta dała wczoraj czekoladki', -0.09526809108697334)
3* - ('judyta dała wczoraj stefanowi czekoladki', -0.09709587511236872)

test: Babuleńka miała dwa rogate koziołki
1* - ('dwa rogate miała babuleńka koziołki', -0.01778379595167186)
2* - ('dwa rogate koziołki miała babuleńka', -0.01778379595167186)
3* - ('koziołki dwa rogate miała babuleńka', -0.01778379595167186)

test: Wczoraj wieczorem spotkałem pewną piękną kobietę
1* - ('pewną piękną kobietę wieczorem spotkałem wczoraj', 0.34293303143335896)
2* - ('wieczorem wczoraj spotkałem pewną piękną kobietę', 0.3394502021434506)
3* - ('spotkałem pewną piękną kobietę wieczorem wczoraj', 0.33941103026045233)

test: Dlaczego zawsze mam największego pecha
1* - ('mam dlaczego zawsze największego pecha', 0.08075399698578717)
2* - ('największego pecha mam dlaczego zawsze', 0.08073122086086013)
3* - ('dlaczego z

## zad3

In [363]:
def load_unigram(file='../train data/1grams'):
    dictionary = {}
    with open(file, 'r', encoding='utf8') as f:
        for line in f:
            line = line.strip().lower()
            line = line.split(' ')
            dictionary[line[1]] = int(line[0])

    return dictionary

def load_tags(file='../train data/supertags.txt'):
    tags = {}
    with open(file, 'r', encoding='utf8') as t:
        for line in t:
            key, value = line.split()
            tags[key.lower()] = value.lower()

    return tags

def tag_mapper(tags, uni):
    dict = defaultdict(list)
    for word in tags:
        weight = 1
        if word in uni:
            weight += uni[word]
        dict[tags[word]].append((word, weight))
    return dict

def find_tag(mapper, w):
    suffix = w if len(w) <= 2 else w[-2:]
    res = 't100'
    best = 0
    for tag in mapper:
        cnt = 0
        ttl = 0
        for word, _ in mapper[tag]:
            suffix2 = word if len(word) <= 2 else w[-2:]
            if suffix == suffix2:
                cnt += 1
            ttl += 1
        if cnt / ttl > best:
            res = tag
            best = cnt / ttl
    return res

def swap_uni(seq, tags, mapper):
    res = []
    for w in seq:
        upper = w[0].isupper()
        w = w.lower()
        if w in tags:
            tag = tags[w]
        else:
            tag = find_tag(mapper, w)
        count = sum([i for _, i in mapper[tag]])
        pick = rnd.randint(0, count)
        new_word = ''
        tmp = 0
        for e, w in mapper[tag]:
            tmp += w
            if pick <= tmp:
                new_word = e
                break
        res.append(new_word if not upper else new_word.capitalize())
    return res

zad3_tags = load_tags()
zad3_uni = load_unigram()
zad3_mapper = tag_mapper(zad3_tags, zad3_uni)

def test_uniswap(tests):
    for phrase in tests:
        print("before:\n{0}\nafter: \n{1}\n".format(phrase, swap_uni(phrase.split(' '), zad3_tags, zad3_mapper)))

In [92]:
tests = [
    "Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę",
    "Duży pies znalazł w śmieciach prawdziwy skarb",
    "Pierwsze co zrobił po tym jak wstał to napił się ciepłej herbaty",
    "Marcin rzekł zmarkniałym głosem, jakie zadanka znowu",
    "Zleziono ostatnio jakieś zdania do napisania",
    "Wczoraj wieczorem zauważyłem dużego dzika w pobliskim lasku"
]
test_uniswap(tests)

before:
Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę
after: 
['Angielski', 'Prokurator', 'zniweczył', 'na', 'niedźwiedziej', 'godzinie', 'nieraz', 'niezliczoną', 'rolę']

before:
Duży pies znalazł w śmieciach prawdziwy skarb
after: 
['Wrześniowy', 'jelonek', 'powitał', 'w', 'latach', 'pylasty', 'domiar']

before:
Pierwsze co zrobił po tym jak wstał to napił się ciepłej herbaty
after: 
['Pierwsze', 'co', 'ukazał', 'na', 'tym', 'jak', 'został', 'to', 'wyjął', 'się', 'salezjańskiej', 'szkoły']

before:
Marcin rzekł zmarkniałym głosem, jakie zadanka znowu
after: 
['Palacz', 'znalazł', 'wystarczy', 'zabraknie', 'jakie', 'biura', 'towarzysko']

before:
Zleziono ostatnio jakieś zdania do napisania
after: 
['Wyzłośliwiono', 'dzisiaj', 'takie', 'podejścia', 'przeciwko', 'życia']

before:
Wczoraj wieczorem zauważyłem dużego dzika w pobliskim lasku
after: 
['Wielokrotnie', 'wywiadem', 'wybrałem', 'ostatniego', 'żółwia', 'w', 'własnym', 'zamachu']



## zad4

In [4]:
def load_digram(file='../train data/2grams', k=10):
    dictionary = defaultdict(list)
    with open(file, 'r', encoding='utf8') as base_vectors_lines:
        for line in base_vectors_lines:
            line = line.strip().lower()
            line = line.split(' ')
            if int(line[0]) >= k:
                dictionary[line[1]].append((line[2], int(line[0])))
            else:
                break

    return dictionary

def tag_mapper_digram(tags, digrams):
    mapper = defaultdict(list)
    for start in digrams:
        if start not in tags:
            continue
        for end, weight in digrams[start]:
            if end not in tags:
                continue
            tag1 = tags[start]
            tag2 = tags[end]
            mapper[(tag1, tag2)].append(((start, end), weight))
    return mapper

zad4_di = load_digram()
zad4_mapper = tag_mapper_digram(zad3_tags, zad4_di)

In [94]:
def bi_swap(words, tags, reversed_tags, reversed_bi_tags, bigrams):
    new_phrase = []
    new_phrase.append(swap_uni(words[0].lower().split(' '), zad3_tags, zad3_mapper)[0].capitalize())
    
    for start, end in zip(words, words[1:]):
        upper = end[0].isupper()
        start = start.lower()
        end = end.lower()
        
        end_tag = tags[end] if end in tags else find_tag(reversed_tags, end)
        
        total = 0
        sub = '#'
        if start in bigrams:
            for extension, weight in bigrams[start]:
                if extension in tags and tags[extension] == end_tag:
                    total += weight

            if total > 0:
                choice = rnd.randint(0, total)

            total = 0
            for extension, weight in bigrams[start]:
                if extension in tags and tags[extension] == end_tag:
                    total += weight
                    if choice <= total:
                        sub = extension
                        break
        if total == 0:
            start = new_phrase[-1]
            if start in bigrams:
                for extension, weight in bigrams[start]:
                    if extension in tags and tags[extension] == end_tag:
                        total += weight

                if total > 0:
                    choice = rnd.randint(0, total)

                total = 0
                for extension, weight in bigrams[start]:
                    if extension in tags and tags[extension] == end_tag:
                        total += weight
                        if choice <= total:
                            sub = extension
                            break

        if total == 0:
            new_phrase.append("|")
            sub = swap_uni(end.split(' '), zad3_tags, zad3_mapper)[0]
        new_phrase.append(sub if not upper else sub.capitalize())

    return new_phrase


def test_diswap(tests):
    for phrase in tests:
        print("before:\n{0}\nafter: \n{1}\n".format(phrase, bi_swap(phrase.split(' '), zad3_tags, zad3_mapper, zad4_mapper, zad4_di)))

In [95]:
test_diswap(tests)

before:
Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę
after: 
['Drobny', 'Chłopczyk', '|', 'wskazał', 'w', 'niedalekiej', 'liczbie', 'około', '|', 'ukrytą', '|', 'poprawkę']

before:
Duży pies znalazł w śmieciach prawdziwy skarb
after: 
['Tradycyjny', 'pies', 'zaatakował', 'w', 'latach', '|', 'pszeniczny', 'popis']

before:
Pierwsze co zrobił po tym jak wstał to napił się ciepłej herbaty
after: 
['Pierwsze', 'co', 'przekazał', 'na', 'tym', 'jak', 'został', '|', 'to', 'ujął', 'się', 'polskiej', 'wody']

before:
Marcin rzekł zmarkniałym głosem, jakie zadanka znowu
after: 
['Chłopak', '|', 'powiedział', '|', 'pozwoli', '|', 'zechce', '|', 'które', 'wrażenia', '|', 'sztywno']

before:
Zleziono ostatnio jakieś zdania do napisania
after: 
['Przystąpiono', '|', 'znacznie', 'jakieś', 'źródła', 'wśród', 'biurka']

before:
Wczoraj wieczorem zauważyłem dużego dzika w pobliskim lasku
after: 
['Chwilowo', 'wieczorem', 'wróciłem', '|', 'strategicznego', '|', 'jelenia', '

## zad5

In [379]:
def PPMI(word, k):
    RE = re.compile(r'[a-zA-ZęóąśłżźćńĘÓĄŚŁŻŹĆŃ\-]+')
    word = word.lower()
    if word not in zad4_di:
        print('Unique word: {1}'.format(k))
        return []
    result = []
    count = sum([v for _, v in zad4_di[word]])
    uni_count = sum([zad3_uni[c] for c in zad3_uni])
    for w, num in zad4_di[word]:
        if w not in zad3_uni:
            continue
        res = re.findall(RE, w)
        if len(res):
            w = res[0]
        if any([v == w for v, _ in result]):
            continue
        result.append((w, np.log((num / count) / ((zad3_uni[word] / uni_count) * (zad3_uni[w] / uni_count)))))
    result.sort(key=lambda x: x[1], reverse=True)
    return result[:k]

In [503]:
test_words = ["kot", "pies", "Polityk", "student", "doktor", "samochód", "praca", "kula"]

P1 = []
P2 = []
P3 = []
for w in test_words:
    P1.append(PPMI(w, 10))
    P2.append(PPMI_ssc(w, 10))
    P3.append(PPMI_T(w, 10))
merged = [(w, list(zip(e1, e2, e3)))for w, e1, e2, e3 in zip(test_words, P1, P2, P3)]

In [504]:
pp = pprint.PrettyPrinter(width=200, depth=7)
pp.pprint(merged)

[('kot',
  [(('napłakał', 24.72022418679312), ('napłakał', 15689.225826636533), ('przybora', 22.12508170698412)),
   (('przybora', 22.12508170698412), ('przybora', 15688.560193068817), ('napłakał', 12.966420183097531)),
   (('domowy', 19.325993853693088), ('domowy', 15677.498209897412), ('domowy', 11.322987245716453)),
   (('wskoczył', 19.249385880237284), ('wskoczył', 15676.5603508274), ('iwan', 9.826378611889014)),
   (('przebiegł', 18.935915604836296), ('przebiegł', 15671.87844035071), ('wskoczył', 7.495581876541694)),
   (('iwan', 18.65717014235092), ('iwan', 15666.285460523497), ('przebiegł', 2.498328300288673)),
   (('leży', 16.258266431211688), ('leży', 15436.119675100197), ('jest', 0.30667368904797243)),
   (('musi', 14.711691525477544), ('musi', 14500.61388179607), ('był', -0.7921593317136839)),
   (('miał', 14.282371937536183), ('miał', 13863.23860749115), ('na', -1.126524754286951)),
   (('ma', 14.216625287825083), ('ma', 13739.150202051314), ('w', -1.1619817957277299))]),
 

In [407]:
test_PPMI(PPMI)

word: kot
[   ('napłakał', 24.72022418679312),
    ('przybora', 22.12508170698412),
    ('domowy', 19.325993853693088),
    ('wskoczył', 19.249385880237284),
    ('przebiegł', 18.935915604836296),
    ('iwan', 18.65717014235092),
    ('leży', 16.258266431211688),
    ('musi', 14.711691525477544),
    ('miał', 14.282371937536183),
    ('ma', 14.216625287825083)]

word: pies
[   ('olejemy', 21.70176720187256),
    ('pogrzebany', 21.489543540672372),
    ('tropiący', 20.93962714982566),
    ('pogryzł', 19.85343738115611),
    ('zaszczekał', 19.729612031043764),
    ('szczekał', 19.533332121999102),
    ('gończy', 19.383433751910374),
    ('pawłowa', 19.073278823606532),
    ('ugryzł', 18.54848865949017),
    ('myśliwski', 18.376388656767755)]

word: Polityk
[   ('prawicy', 18.306191839255717),
    ('platformy', 17.71085994371281),
    ('pis', 17.556616161239816),
    ('sojuszu', 17.482028246315437),
    ('sld', 17.195970147024074),
    ('powinien', 16.73304277189076),
    ('partii', 16.51

In [392]:
def PPMI_ssc(word, k):
    RE = re.compile(r'[a-zA-ZęóąśłżźćńĘÓĄŚŁŻŹĆŃ\-]+')
    word = word.lower()
    if word not in zad4_di:
        print('Unique word: {1}'.format(word))
        return []
    result = []
    count = sum([v for _, v in zad4_di[word]])
    uni_count = sum([zad3_uni[c] for c in zad3_uni])
    for w, num in zad4_di[word]:
        if w not in zad3_uni:
            continue
        res = re.findall(RE, w)
        if len(res):
            w = res[0]
        if any([v == w for v, _ in result]):
            continue
        x = (num / uni_count)
        result.append((w, (x - ((zad3_uni[word] / uni_count) * (zad3_uni[w] / uni_count))) / np.sqrt(np.power(x * (1.0 - x), 2) / uni_count)))
    result.sort(key=lambda x: x[1], reverse=True)
    return result[:k]

In [408]:
test_PPMI(PPMI_ssc)

word: kot
[   ('napłakał', 15689.225826636533),
    ('przybora', 15688.560193068817),
    ('domowy', 15677.498209897412),
    ('wskoczył', 15676.5603508274),
    ('przebiegł', 15671.87844035071),
    ('iwan', 15666.285460523497),
    ('leży', 15436.119675100197),
    ('musi', 14500.61388179607),
    ('miał', 13863.23860749115),
    ('ma', 13739.150202051314)]

word: pies
[   ('olejemy', 15688.833790130051),
    ('pogrzebany', 15688.731323412085),
    ('tropiący', 15688.327839350364),
    ('pogryzł', 15686.462334578868),
    ('zaszczekał', 15686.090426552846),
    ('szczekał', 15685.399450777399),
    ('gończy', 15684.772517959933),
    ('pawłowa', 15683.134585735652),
    ('ugryzł', 15678.896024882824),
    ('myśliwski', 15676.946450103364)]

word: Polityk
[   ('prawicy', 15651.423155144634),
    ('platformy', 15620.624536284018),
    ('pis', 15609.175439421197),
    ('sojuszu', 15602.972113017071),
    ('sld', 15574.391119211406),
    ('powinien', 15506.757203065756),
    ('partii', 1

In [501]:
t_sum = defaultdict(int)
for w in zad3_uni:
    if w in zad3_tags:
        t_sum[zad3_tags[w]] += 1

def PPMI_tags(word, w2):
    result = []

    t1 = zad3_tags[word]
    
    count = sum([sum([x[1] for x in zad4_mapper[w]]) for w in zad4_mapper])
    t_count = sum([1 for c in zad3_tags])
    if w2 not in zad3_tags:
        return 0
    t2 = zad3_tags[w2]
    if (t1, t2) not in zad4_mapper:
        return 0
    weight = sum([x[1] for x in zad4_mapper[(t1, t2)]])
    return (np.log((weight / count) / (t_sum[t1] / t_count) * (t_sum[t2] / t_count)))

def PPMI_T(word, k):
    RE = re.compile(r'[a-zA-ZęóąśłżźćńĘÓĄŚŁŻŹĆŃ]+')
    word = word.lower()
    if word not in zad4_di:
        print('Unique word: {1}'.format(k))
        return []
    result = []
    count = sum([v for _, v in zad4_di[word]])
    uni_count = sum([zad3_uni[c] for c in zad3_uni])
    for w, num in zad4_di[word]:
        if w not in zad3_uni:
            continue
        res = re.findall(RE, w)
        if len(res):
            w = res[0]
        else:
            continue
        if any([v == w for v, _ in result]):
            continue
        r = PPMI_tags(word, w)
        result.append((w, np.log((num / count) / ((zad3_uni[word] / uni_count) * (zad3_uni[w] / uni_count))) + r))
    result.sort(key=lambda x: x[1], reverse=True)
    return result[:k]

In [502]:
PPMI_T("kot", 10)

[('przybora', 22.12508170698412),
 ('napłakał', 12.966420183097531),
 ('domowy', 11.322987245716453),
 ('iwan', 9.826378611889014),
 ('wskoczył', 7.495581876541694),
 ('przebiegł', 2.498328300288673),
 ('jest', 0.30667368904797243),
 ('był', -0.7921593317136839),
 ('na', -1.126524754286951),
 ('w', -1.1619817957277299)]

## zad 6

In [329]:
PT_lines = []
RE = re.compile(r'[a-zA-ZęóąśłżźćńĘÓĄŚŁŻŹĆŃ\-]+')
with open('../train data/pan-tadeusz.txt', 'r', encoding='utf8') as pt:
    nf = False
    for line in pt:
        if not line.strip():
            continue
        if not nf:
            if line.strip().split(' ')[0] == 'Litwo!':
                nf = True
        if nf:
            if line.strip()[0] == '-':
                break
            else:
                PT_lines.append([word.lower() for word in re.findall(RE, line)])
                
vovels = ['a', 'ą', 'e', 'ę', 'i', 'o', 'u', 'y', 'ó']
def accent(sentence):
    res = []
    for w in sentence:
        c = 0
        for c1, c2 in zip('^' + w, w):
            if c2 in vovels and c1 != 'i':
                c += 1
        res.append(c)
    return res

word_to_tag = {}
tag_to_word = defaultdict(list)
with open('../train data/supertags.txt', 'r', encoding='utf8') as file:
    for line in file:
        word, tag = line.split()
        word_to_tag[word] = tag
        tag_to_word[tag].append(word)

def get_rhyme(word):
    for i in range(1, len(word)):
        if accent([word[-i:]])[0] == 2:
            return word[-i:]
    return None

def single_verset(sentences):
    idx = rnd.randint(1, len(sentences) - 2)
    rhyme = get_rhyme(sentences[idx][-1])

    if get_rhyme(sentences[idx-1][-1]) == rhyme:
        return sentences[idx-1], sentences[idx]
    elif get_rhyme(sentences[idx+1][-1]) == rhyme:
        return sentences[idx], sentences[idx+1]
    return None, None

In [435]:
def same_tags(w):
    if w in word_to_tag and rnd.random() > 0.05:
        return tag_to_word[word_to_tag[w]]
    elif ('#' + w)[-3:] in word_to_tag and rnd.random() > 0.05:
        return tag_to_word[word_to_tag[('#' + w)[-3:]]]
    else:
        return tag_to_word[rnd.choice(list((word_to_tag.values())))]

def random_werse():    
    while True:
        sentence1 = []
        sentence2 = []
        start = rnd.randint(0, len(PT_lines)-2)
        w1, w2 = single_verset(PT_lines)
        if not w1:
            continue
    
        if get_rhyme(w1[-1]) != get_rhyme(w2[-1]):
            continue    

        a1, a2 = accent(w1), accent(w2)
        
        t1 = [same_tags(w) for w in w1]
        t2 = [same_tags(w) for w in w2]

        if None in t1 or None in t2:
            continue
        
        candidates1 = [[c for c in candidate if accent([c])[0] == a] for candidate, a in zip(t1, a1)]
        candidates2 = [[c for c in candidate if accent([c])[0] == a] for candidate, a in zip(t2, a2)]

        cn1 = []
        cn2 = []
        
        for w in candidates1[-1]:
            for c2 in candidates2[-1]:
                if get_rhyme(w) == get_rhyme(c2):
                    cn1.append(w)
                    break
        for w in candidates2[-1]:
            for c2 in candidates1[-1]:
                if get_rhyme(w) == get_rhyme(c2):
                    cn2.append(w)
                    break

        candidates1[-1] = cn1
        candidates2[-1] = cn2

        if [] in candidates1 or [] in candidates2:
            continue    
            
        weights1 = [[zad3_uni[w] if w in zad3_uni else 2 for w in l] for l in candidates1]
        weights2 = [[zad3_uni[w] if w in zad3_uni else 2 for w in l] for l in candidates2]

        for _ in range(1000):
            res1 = []
            res2 = []
            for wl, weights in zip(candidates1, weights1):
                k = []
                ww = []
                for x, w2 in zip(wl, weights):
                    if x in res1 or x in res2:
                        break
                    else:
                        k.append(x)
                        ww.append(w2)
                if not k:
                    break
                res1.append(rnd.choices(k, weights=ww)[0])
    
            for wl, weights in zip(candidates2, weights2):
                k = []
                ww = []
                for x, w2 in zip(wl, weights):
                    if x in res2 or x in res1:
                        break
                    else:
                        k.append(x)
                        ww.append(w2)
                if not k:
                    break

                res2.append(rnd.choices(k, weights=ww)[0])
            
            if len(res1) < len(candidates1) or len(res2) < len(candidates2):
                continue
            if get_rhyme(res1[-1]) != get_rhyme(res2[-1]):
                continue
            return "{0}\n{1}\n".format(" ".join(res1), " ".join(res2))
        
    return ""

In [437]:
print(random_werse())

czyli imprezy łamiąc telefony
gdy zablokują wyświetlane strony



In [438]:
print(random_werse())

obok na godnej drodze internat hrabiowski
po ciemnej wziął psułoby pół i jarzembowski



In [439]:
print(random_werse())

wprawdzie dała się mało sensem nabożniejsza
bo zamknięta a obiekt oznacza i zmniejsza



In [440]:
print(random_werse())

dość nie dzieli ta śląska wyniszczenie babia
a potem czegóż jeszcze zbył mi tak ten hrabia



In [441]:
print(random_werse())

bobak wciąż mówił a raz coraz więcej nosił
aż budownictwo szkołą uporem poprosił



In [442]:
print(random_werse())

dojrzawszy zadam miejsca rosną po moskali
bezpośrednio ich maluch z twórcą kierowali



In [443]:
print(random_werse())

i choć jej zamierzają służbowe siatkarki
nie mogła w który sposób wrócić gospodarki



In [436]:
print(random_werse())

że stali daną siebie oba urzędnicy
i przez naszej wybili razem kamienicy



## zad 7

In [371]:

PPMI_ucount = sum([zad3_uni[c] for c in zad3_uni])

def PPMI_value(w1, w2, sb=0):
    if sb == 3:
        return 0.0
    if w1 not in zad3_uni or w2 not in zad3_uni:
        return PPMI_value(superbase_mapping([w1])[0], superbase_mapping(w2)[0], sb + 1)
    p = False
    for w, _ in zad4_di[w1]:
        if w == w2:
            p = True
            break
    if not p:
        return PPMI_value(w1, superbase_mapping([w2])[0], sb + 1)
    num = 0
    PPMI_count = sum([v for _, v in zad4_di[w1]])
    for w, n in zad4_di[w1]:
        if w == w2:
            num = n
            break
    return np.log((num / PPMI_count) / ((zad3_uni[w1] / PPMI_ucount) * (zad3_uni[w2] / PPMI_ucount)))

def PPMI_value_sentence(sentence):
    score = 0.0
    for w1, w2 in zip(sentence, sentence[1:]):
        score += PPMI_value(w1, w2)
    return score
        

In [433]:
def same_tags(w):
    if w in word_to_tag and rnd.random() > 0.05:
        return tag_to_word[word_to_tag[w]]
    elif ('#' + w)[-3:] in word_to_tag and rnd.random() > 0.05:
        return tag_to_word[word_to_tag[('#' + w)[-3:]]]
    else:
        return tag_to_word[rnd.choice(list((word_to_tag.values())))]

def random_werse_PPMI():    
    while True:
        sentence1 = []
        sentence2 = []
        start = rnd.randint(0, len(PT_lines)-2)
        w1, w2 = single_verset(PT_lines)
        if not w1:
            continue
            
        if get_rhyme(w1[-1]) != get_rhyme(w2[-1]):
            continue
        
        a1, a2 = accent(w1), accent(w2)
        
        t1 = [same_tags(w) for w in w1]
        t2 = [same_tags(w) for w in w2]

        if None in t1 or None in t2:
            continue
        
        candidates1 = [[c for c in candidate if accent([c])[0] == a] for candidate, a in zip(t1, a1)]
        candidates2 = [[c for c in candidate if accent([c])[0] == a] for candidate, a in zip(t2, a2)]

        cn1 = []
        cn2 = []
        
        for w in candidates1[-1]:
            for c2 in candidates2[-1]:
                if get_rhyme(w) == get_rhyme(c2):
                    cn1.append(w)
                    break
        for w in candidates2[-1]:
            for c2 in candidates1[-1]:
                if get_rhyme(w) == get_rhyme(c2):
                    cn2.append(w)
                    break

        candidates1[-1] = cn1
        candidates2[-1] = cn2

        if [] in candidates1 or [] in candidates2:
            continue    
            
        weights1 = [[zad3_uni[w] if w in zad3_uni else 2 for w in l] for l in candidates1]
        weights2 = [[zad3_uni[w] if w in zad3_uni else 2 for w in l] for l in candidates2]

        results_final1 = []
        results_final2 = []
        for _ in range(20000):
            res1 = []
            res2 = []
            for wl, weights in zip(candidates1, weights1):
                k = []
                ww = []
                for x, w2 in zip(wl, weights):
                    if x in res1 or x in res2:
                        break
                    else:
                        k.append(x)
                        ww.append(w2)
                if not k:
                    break
                res1.append(rnd.choices(k, weights=ww)[0])
    
            for wl, weights in zip(candidates2, weights2):
                k = []
                ww = []
                for x, w2 in zip(wl, weights):
                    if x in res2 or x in res1:
                        break
                    else:
                        k.append(x)
                        ww.append(w2)
                if not k:
                    break

                res2.append(rnd.choices(k, weights=ww)[0])
            
            if len(res1) < len(candidates1) or len(res2) < len(candidates2):
                continue
            if get_rhyme(res1[-1]) != get_rhyme(res2[-1]):
                continue
            results_final1.append((res1, PPMI_value_sentence(res1)))
            results_final2.append((res2, PPMI_value_sentence(res2)))
        
        if len(results_final1) < 5:
            continue

        res_final = []
        bs = 0.0
        for (r1, s1), (r2, s2) in zip(results_final1, results_final2):
            res_final.append((s1 + s2, (r1, r2)))
        res_final.sort(key=lambda x: x[0], reverse=True)
        return ["score {0}\n{1}\n{2}\n\n".format(x[0], " ".join(x[1][0]), " ".join(x[1][1])) for x in res_final[:5]]

In [375]:
res = random_werse_PPMI()
for w in res:
    print(w)

score 30.581140223145646
źle stąd bo duchom jezus nas mdły odpoczynek
kroćset trocin wyników wód przeszkód dożynek


score 30.333791190656523
dziś stąd bo diabłom jezus nas mdły wypoczynek
kroćset perfum rodzajów tam siedzib dożynek


score 24.570426620903177
znów stąd bo ciskom jezus nas pstry wypoczynek
kroćset dziejów tysięcy stron książek dożynek


score 24.503410873358373
dziś stąd bo koniom ojciec nas mdły odpoczynek
kroćset dziejów obrazów tam osób dożynek


score 23.21817862916079
wraz stąd bo koniom anioł nas mdły wypoczynek
kroćset swarów zespołów szkół zasad dożynek




In [377]:
res = random_werse_PPMI()
for w in res:
    print(w)

score 69.8817194313011
była to katastrofa osiecka warszawa
pod której miał pójść nawet nowy tekst ustawa


score 63.16879810000797
była to koalicja mołdawska warszawa
po której miał pójść nieco własny rząd ustawa


score 61.67844223024633
była to sytuacja winnicka warszawa
na której miał wejść dzisiaj cały sprzęt ustawa


score 61.172181647032204
była to propozycja balicka christina
po której miał wyjść wreszcie każdy rząd rodzina


score 58.17886660195677
była to medycyna wileńska połowa
na żadnej miał wejść nawet mały skok rozmowa




In [434]:
res = random_werse_PPMI()
for w in res:
    print(w)

score 58.00076940509564
polski dzisiejszy nawet rada cześć kordiana
poprawna i już wielkim znakiem obtrząsana


score 56.54171839496032
polski dzisiejszy trudno praca rzecz bogdana
bezmyślna i już wielkim wpływem nazywana


score 56.52177469230555
polski powyższy przedtem kilka wieś szymona
rozsądna i już wielkim hukiem ulubiona


score 50.33166632029846
drogi silniejszy nawet praca rzecz stefana
ostatnia i już nowym typem obstawiana


score 49.590862979220454
drogi wczorajszy wczoraj hanna maź romana
prawdziwa i już głównym celem obniżana


